In [1]:
import sqlite3
import os
from datetime import datetime
from datetime import timedelta
from sklearn.datasets import fetch_20newsgroups
import time
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pickle
from threading import Thread
from sklearn.model_selection import train_test_split
from collections import Counter


start_time = time.time()

twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
cat = list(twenty_train.target_names)


def writeToFile(fName,message):
    file = open(fName,"a")
    file.write (message)
    file.close()
    

def savePickleFile(d,varList):
    with open(d, 'wb') as f: 
        pickle.dump(varList, f)
    f.close()
    
def loadPickleFile(d):
    with open(d,'rb') as f:  # Python 3: open(..., 'rb')
        varList = pickle.load(f)
    f.close()
    return varList 

def makeDirectory(folder_name1,sub_folder):
    directory = os.path.join(folder_name1 , sub_folder)
    if not os.path.exists(directory):
            os.makedirs(directory)
    return directory

def getParentDir(cwd):
    parentDir = os.path.dirname(cwd)
    return parentDir

def connectDB():
    cwd = os.getcwd()
    parentDir = os.path.dirname(cwd)
    sqlite_file = parentDir+"/mydatabase.db"
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    command="CREATE TABLE IF NOT EXISTS Stock(date TEXT, stockName TEXT, open INTEGER, close INTEGER, volume INTEGER, change INTEGER, UNIQUE(date, stockName) ON CONFLICT REPLACE);"
    c.execute(command)
    return conn

def extractDB(table_name,predicted,c,company):
    #c is connector SQL
    for i in range(len(data)):
        values = (predicted[i][0],company,data['open'].values[i],
                  data['close'].values[i],data['volume'].values[i],predicted[i][1])
        c.execute("INSERT INTO {tn} VALUES(?,?,?,?,?,?)".format(tn=table_name),values)

def fetchStock(start,end,conn,stockName):
    tableName="Stock"
    c = conn.cursor()
    startDate = start.strftime('%Y%m%d')
    endDate = end.strftime('%Y%m%d')
    command= "select * from %s where substr(date,1,4)||substr(date,6,2)||substr(date,9,2) BETWEEN '%s' and '%s' AND stockName='%s'"%(tableName,startDate,endDate,stockName)
#     print (command)
    c.execute(command)
    result_set = c.fetchall()
#     print (result_set)
    return result_set

def fetchTweets(start,end,conn):
    tableName="Twitter"
    c = conn.cursor()
#     t = datetime(2012, 2, 23, 0, 0)
#     t.strftime('%Y%m%d')
    startDate = start.strftime('%Y%m%d')
    endDate = end.strftime('%Y%m%d')
    command= "select * from %s where substr(date,1,4)||substr(date,6,2)||substr(date,9,2) BETWEEN '%s' and '%s'"%(tableName,startDate,endDate)
    c.execute(command)
    result_set = c.fetchall()
    return result_set

def getAllStockNames(conn):
    tableName="Stock"
    c = conn.cursor()
    command = "SELECT DISTINCT stockName FROM %s"%(tableName)
    c.execute(command)
    result_set = c.fetchall()
    return result_set

def disconnectDB(conn):
    conn.close()
    
def getPricesForStock(name,startObj,endObj):
    conn =connectDB()
    stockValues=fetchStock(startObj,endObj,conn,name)
#     print (stockValues)
    disconnectDB(conn)
    #     a = np.matrix(stockValues)
    return stockValues
def getTweets(startObj,endObj):
    conn =connectDB()
    tweets=fetchTweets(startObj,endObj,conn)
#     print (stockValues)
    disconnectDB(conn)
    #     a = np.matrix(stockValues)
    return tweets
def getXVector(endObj,daysFromThePast):
    global cat;
    startObj = endObj - timedelta(days=daysFromThePast)
    a=getTweets(startObj,endObj)
#     print(cat)
    xVector=[0 for i in range(len(cat))]
#     print(xVector)
    for i in a:
        xVector[cat.index(i[5])]+=1
#     print (xVector)
    sum = 0
    for i in xVector:
        sum+=i
    for i in range(len(xVector)):
        xVector[i]=xVector[i]/sum
#     print (xVector)
    return xVector

#temp fix

def getYVector(name,startObj,endObj):
    a = getPricesForStock(name,startObj,endObj)
    fluctuationsList = []
    dateList=[]
    for i in  a:
        if (i[0]=='2017-08-07'):
            continue;
        fluctuationsList.append(i[-1])
        dateList.append(i[0])
    return dateList,fluctuationsList

#good version
# def getYVector(name,startObj,endObj):
#     a = getPricesForStock(name,startObj,endObj)
#     fluctuationsList = []
#     dateList=[]
#     for i in  a:
#         fluctuationsList.append(i[-1])
#         dateList.append(i[0])
#     return dateList,fluctuationsList

def getDateObj(s):
    datetime_object = datetime.strptime(s, '%Y-%m-%d')
    return datetime_object

def getXandY(name,startObj,endObj,d):
    daysInThePast = d
    dateList,yVector=getYVector(name,startObj,endObj)
    xList = []
    yList = yVector
    for i in dateList:
        xList.append(getXVector(getDateObj(i),daysInThePast))
    return xList,yList

def getXandYMulti(stockArr,startObj,endObj,d):
    yMatrix=[]
    xBuilt=0
    for s in stockArr:
        if (xBuilt==0):
            xVector,yVector=getXandY(s,startObj,endObj,d)
            yMatrix.append(yVector)
        else:
            d,yVector=getYVector(s,startObj,endObj)
            yMatrix.append(yVector)
        
        xBuilt=1
    return xVector,yMatrix
        
def investWithThreshold(yM,threshold):
    for i in range(len(yM)):
        for j in range(len(yM[i])):
            value = float(yM[i][j])

            if (value >= threshold):
                c ="Gain"
            elif (value <= (-1*threshold)):
                c = "Loss"
            else:
                c = "Neutral"
                
            yM[i][j]=c
    return yM

def runMLModel(xV,yV,stockName,d,threshold):
    k='sigmoid'
    clf = svm.SVC(decision_function_shape='ovo',kernel=k)
#     scores = cross_val_score(clf, xV, yV, cv=10)
#     message = ("StockName: %s"%stockName) +"_Days_%s"%str(d) + "_Kernel_%s"%k + "\n"
#     message += ("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)+"\n")
#     X_train, X_test, y_train, y_test = train_test_split( xV, yV, test_size=0.20, random_state=42)
    clf.fit(xV,yV)
    
    OriginPath = getParentDir(os.getcwd())
    subFolderPath=makeDirectory(OriginPath,"PredictionOutput")
    pickleObjPath = makeDirectory(subFolderPath,"MLModels")
    
    pklFName = pickleObjPath + "/" + stockName +"_Days_%s"%str(d) + "_Kernel_%s"%k + ".plk"
    savePickleFile(pklFName,[clf])
    
    reportName = subFolderPath + "/Report.txt"
#     writeToFile(reportName,message)

##---Using to get the score for realiable stock
def runMLModel1(xV,yV,stockName,d,threshold):
    k='sigmoid'
    clf = svm.SVC(decision_function_shape='ovo',kernel=k)
    scores = cross_val_score(clf, xV, yV, cv=10)
    message = ("StockName: %s"%stockName) +"_Days_%s"%str(d) + "_Kernel_%s"%k + "_Thres_%s"%str(threshold) + "\n"
    message += ("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)+"\n")
    message +="\n"
    
    OriginPath = getParentDir(os.getcwd())
    subFolderPath=makeDirectory(OriginPath,"PredictionOutput")
    pickleObjPath = makeDirectory(subFolderPath,"MLModels")
    
    pklFName = pickleObjPath + "/" + stockName +"_Days_%s"%str(d) + "_Kernel_%s"%k + ".plk"
    savePickleFile(pklFName,[clf])
    
    reportName = subFolderPath + "/Report.txt"
    writeToFile(reportName,message)


In [2]:
pickedStock = ["AAPL","AMZN","FB","JPM","TSLA","BDX","GOOGL","PGR","JNJ","CSCO","CLX","PG","MSFT"
               ,"CHKP","ORCL","DIS","SAFT","ITC",
              "KO","WMT","VZ","TRV","MCD","COST","NFLX","TWTR"]

# conn =connectDB()
days = 10
if (os.path.isfile("/home/ubun/Desktop/StockPre/PredictionOutput/MLModels/yMatrix.plk")):
    ymatrix = loadPickleFile("/home/ubun/Desktop/StockPre/PredictionOutput/MLModels/yMatrix_%s.plk"%(days))[0]
    xV = loadPickleFile("/home/ubun/Desktop/StockPre/PredictionOutput/MLModels/xVector_%s.plk"%(days))[0]
else:
    xV,ymatrix = getXandYMulti(pickedStock,datetime(2016,1,1),datetime(2018,3,20),days)
    savePickleFile("/home/ubun/Desktop/StockPre/PredictionOutput/MLModels/xVector_%s.plk"%(days),[xV])
    savePickleFile("/home/ubun/Desktop/StockPre/PredictionOutput/MLModels/yMatrix_%s.plk"%(days),[ymatrix])

def getPickelFile(path,threshold):
    if (os.path.isfile(path)):
        yM = loadPickleFile(path)[0]
        return yM
    else:
        yM = investWithThreshold(ymatrix,threshold)
        savePickleFile(path,[yM])
        return yM
    
def buildModles(xV,ymatrix,days,threshold):
    path = "/home/ubun/Desktop/StockPre/PredictionOutput/MLModels/yM_Days_%d_Thres_%d.plk"%(days,threshold)
    yM = getPickelFile(path,threshold)
    
    # for s in pickedStock:
    #     runMLModel(xV,yM[pickedStock.index(s)],s,days)



    print ("--- Just Finish Building X---")
    print ("--- %s seconds ----" % (time.time() - start_time))

    threads=[]
    for s in pickedStock:
        t = Thread(target = runMLModel1, args = (xV,yM[pickedStock.index(s)],s,days,threshold,))
#         t1 =Thread(target = runMLModel1, args = (xV,yM[pickedStock.index(s)],s,days,threshold,))
    #     t = Thread(target = getTweets, args = (p,)) 
        threads.append(t)
#         threads.append(t1)
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    return yM

yM = buildModles(xV,ymatrix,days,0.5)

print("Exit main thread")
print("--- %s seconds ---" % (time.time() - start_time))


--- Just Finish Building X---
--- 367.5068292617798 seconds ----


/home/ubun/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested below threads, setting n_jobs=1
  **self._backend_args)
/home/ubun/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested below threads, setting n_jobs=1
  **self._backend_args)
/home/ubun/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested below threads, setting n_jobs=1
  **self._backend_args)
/home/ubun/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested below threads, setting n_jobs=1
  **self._backend_args)
/home/ubun/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loop

Exit main thread
--- 369.6509132385254 seconds ---


In [3]:
# fb=getYVector("FB",datetime(2016,1,1),datetime(2018,3,20))
# aapl=getYVector("AAPL",datetime(2016,1,1),datetime(2018,3,20))

In [4]:
# set(fb[0])-set(aapl[0])
def preDict(xVector,stockname):
    data= [xVector]
    k="sigmoid"
    d=10
    OriginPath = getParentDir(os.getcwd())
    subFolderPath=makeDirectory(OriginPath,"PredictionOutput")
    pickleObjPath = makeDirectory(subFolderPath,"MLModels")
    
    pklFName = pickleObjPath + "/" + stockname +"_Days_%s"%str(d) + "_Kernel_%s"%k + ".plk"
    if (os.path.isfile(pklFName)):
        clf=loadPickleFile(pklFName)[0]
        print (stockname + " " + clf.predict(data)[0])
    else:
        print ("This %s is not avalable"%stockname)


In [5]:
def todayPrediction(pickedStock,days):
    today = datetime.now() - timedelta(days=0)
    a = getXVector(today,days)
    for s in pickedStock:
        preDict(a,s)

# todayPrediction(pickedStock,days)

AAPL Neutral
AMZN Gain
FB Neutral
JPM Neutral
TSLA Gain
BDX Neutral
GOOGL Neutral
PGR Neutral
JNJ Neutral
CSCO Neutral
CLX Neutral
PG Neutral
MSFT Neutral
CHKP Neutral
ORCL Neutral
DIS Neutral
SAFT Neutral
This ITC is not avalable
KO Neutral
WMT Neutral
VZ Neutral
This TRV is not avalable
MCD Neutral
COST Neutral
NFLX Gain
TWTR Gain

dict_values([235, 138, 182])